# Fiona and Shapely

[Fiona](http://toblerity.org/fiona/) is build on top of OGR and it introduces the object-oriented approach to the work with vector data. The data are re-mapped in to [GeoJSON](http://geojson.org) structure. The interaction is more straight-forward compared to OGR.

Fiona is not suited to be the best for all tasks - simplicity is buyed back by the fact, all data need to be loaded in memmory. Also speed can be lower, when dealing with larger amounts of data. Where OGR uses pointers, Fiona has to deal with whole python objects. Generally: if you need to filter vector features, OGR should be faster, if you need to work with features sequentially, Fiona should be better.

**Note**:
> Fiona is great for binary data formats. For JSON, use the `json` python package. For data stored in database, use standard Python database interface.

[Shapely](http://toblerity.org/shapely/manual.html) Shapely is a Python package for set-theoretic analysis and manipulation of planar features using (via Python’s ctypes module) functions from the well known and widely deployed GEOS library. 

**Short version:** 
> Fiona is here for attribute and whole data source manipulation, Shapely just manupulates vector data geometries.

## Open data source

First we need to create so called collection of features:

In [3]:
import fiona
chko = fiona.open('../../data/protected_areas-etrs.shp', 'r') # collection - data are stored in ../../data directory 
chko

<open Collection '../../data/protected_areas-etrs.shp:protected_areas-etrs', mode 'r' at 0x7fdda48ca1d0>

We can now mine some metadata about this feature collection, see [documentation](http://toblerity.org/fiona/manual.html)

In [4]:
# driver
chko.driver

'ESRI Shapefile'

In [5]:
# coordinate reference system
chko.crs

{'ellps': 'GRS80',
 'lat_0': 52,
 'lon_0': 10,
 'no_defs': True,
 'proj': 'laea',
 'units': 'm',
 'x_0': 4321000,
 'y_0': 3210000}

In [6]:
# file name
chko.path

'../../data/protected_areas-etrs.shp'

In [7]:
# layer name
chko.name

'protected_areas-etrs'

In [8]:
# bounding box coordinates
chko.bounds

(4493979.359512844, 2839119.848736721, 4952710.629522002, 3106267.3238070104)

In [9]:
# everything in one step
print(chko.meta)

{'driver': 'ESRI Shapefile', 'schema': {'properties': OrderedDict([('gml_id', 'str:80'), ('OBJECTID', 'int:10'), ('KOD', 'int:10'), ('KAT', 'str:4'), ('NAZEV', 'str:27'), ('ZONA', 'str:3'), ('ROZL', 'float:24.15'), ('OP_TYP', 'str:3'), ('IUCN', 'str:2'), ('ZMENA_G', 'int:10'), ('ZMENA_T', 'int:10'), ('PREKRYV', 'int:10'), ('SHAPE.AREA', 'float:24.15'), ('SHAPE.LEN', 'float:24.15')]), 'geometry': 'Polygon'}, 'crs': {'proj': 'laea', 'lat_0': 52, 'lon_0': 10, 'x_0': 4321000, 'y_0': 3210000, 'ellps': 'GRS80', 'units': 'm', 'no_defs': True}, 'crs_wkt': 'PROJCS["Lambert_Azimuthal_Equal_Area",GEOGCS["GCS_GRS 1980(IUGG, 1980)",DATUM["unknown",SPHEROID["GRS80",6378137,298.257222101]],PRIMEM["Greenwich",0],UNIT["Degree",0.017453292519943295]],PROJECTION["Lambert_Azimuthal_Equal_Area"],PARAMETER["latitude_of_center",52],PARAMETER["longitude_of_center",10],PARAMETER["false_easting",4321000],PARAMETER["false_northing",3210000],UNIT["Meter",1]]'}


In [10]:
# try better format
import json
print(json.dumps(chko.meta, sort_keys=True, indent=4, separators=(',', ': ')))

{
    "crs": {
        "ellps": "GRS80",
        "lat_0": 52,
        "lon_0": 10,
        "no_defs": true,
        "proj": "laea",
        "units": "m",
        "x_0": 4321000,
        "y_0": 3210000
    },
    "crs_wkt": "PROJCS[\"Lambert_Azimuthal_Equal_Area\",GEOGCS[\"GCS_GRS 1980(IUGG, 1980)\",DATUM[\"unknown\",SPHEROID[\"GRS80\",6378137,298.257222101]],PRIMEM[\"Greenwich\",0],UNIT[\"Degree\",0.017453292519943295]],PROJECTION[\"Lambert_Azimuthal_Equal_Area\"],PARAMETER[\"latitude_of_center\",52],PARAMETER[\"longitude_of_center\",10],PARAMETER[\"false_easting\",4321000],PARAMETER[\"false_northing\",3210000],UNIT[\"Meter\",1]]",
    "driver": "ESRI Shapefile",
    "schema": {
        "geometry": "Polygon",
        "properties": {
            "IUCN": "str:2",
            "KAT": "str:4",
            "KOD": "int:10",
            "NAZEV": "str:27",
            "OBJECTID": "int:10",
            "OP_TYP": "str:3",
            "PREKRYV": "int:10",
            "ROZL": "float:24.15",
       

## Features in collections
Features within opened collection can be iterated

In [12]:
print(len(chko))

5626


## Coordinate reference systems
Fiona comes with primitive tools for dealing with coordinate reference systems, with GDAL library in the background

In [22]:
natural = fiona.open('../../data/protected_areas-etrs.shp', 'r')
from fiona.crs import to_string
print(to_string(natural.crs))

+init=epsg:4326


Simillary, new CRS definition can be created

In [13]:
from fiona.crs import from_epsg
from_epsg(3857)

{'init': 'epsg:3857', 'no_defs': True}

## Walking through features
we can either iterate through available features

In [14]:
for feature in chko[0:10]:
    print(feature['geometry']['type'])

Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon
Polygon


or particular feature can be choosed

In [15]:
print(chko[54]['properties']['NAZEV'])

ÄeskÃ½ rÃ¡j


## Feature geometry and shapely library
[Shapely](http://toblerity.org/shapely) converts feature geometry into GeoJSON structure. It also contains tools for geometry manipulations

In [16]:
from shapely.geometry import shape
cr = chko[54]
poly = shape(cr['geometry'])
poly.bounds

(4685576.577618335, 3067490.2318713292, 4687748.187993193, 3069132.552762671)

Now we can either generalize given geometry or create buffer

In [18]:
simple = poly.simplify(10)
simple.intersects(poly)
buff = poly.buffer(10)
buff.contains(poly)

True

some feature attributes can be fixed as well, at the end, we write new created feature down

In [ ]:
from shapely.geometry import mapping
feature = cr.copy()
feature.update(id=-1, geometry=mapping(buff))
feature["properties"].update(NAZEV="Mordor")
print(feature["properties"].keys())
print(chko.schema["properties"].keys())
chko = fiona.open('../../data/protected_areas-etrs.shp', 'a')
print("Features before: %d " % len(chko))
chko.write(feature)
print("Features after: %d " % len(chko))
chko.close()

---
[<- Introduction to vectors](00_introduction.ipynb) | [GDAL ->](02_ogr.ipynb)